In [5]:
from pulp import LpMaximize, LpProblem, LpVariable

# Define the problem
problem = LpProblem("Maximize_Revenue", LpMaximize)

# Decision variables
x1 = LpVariable("First_Class_Seats", lowBound=0, cat="Integer")
x2 = LpVariable("Business_Class_Seats", lowBound=0, cat="Integer")
x3 = LpVariable("Economy_Class_Seats", lowBound=0, cat="Integer")

# Auxiliary variables for demand-limited seats
d1_high = LpVariable("First_Class_High_Demand", lowBound=0, cat="Integer")
d2_high = LpVariable("Business_Class_High_Demand", lowBound=0, cat="Integer")
d3_high = LpVariable("Economy_Class_High_Demand", lowBound=0, cat="Integer")

d1_med = LpVariable("First_Class_Med_Demand", lowBound=0, cat="Integer")
d2_med = LpVariable("Business_Class_Med_Demand", lowBound=0, cat="Integer")
d3_med = LpVariable("Economy_Class_Med_Demand", lowBound=0, cat="Integer")

d1_low = LpVariable("First_Class_Low_Demand", lowBound=0, cat="Integer")
d2_low = LpVariable("Business_Class_Low_Demand", lowBound=0, cat="Integer")
d3_low = LpVariable("Economy_Class_Low_Demand", lowBound=0, cat="Integer")

# Objective function: Expected Revenue
problem += (
    0.4 * (1600 * d1_high + 1100 * d2_high + 500 * d3_high) +
    0.3 * (1600 * d1_med + 1100 * d2_med + 500 * d3_med) +
    0.3 * (1600 * d1_low + 1100 * d2_low + 500 * d3_low)
)

# Capacity constraint
problem += 2 * x1 + 1.5 * x2 + x3 <= 190

# Demand constraints for high demand
problem += d1_high <= 25
problem += d2_high <= 60
problem += d3_high <= 210
problem += d1_high <= x1
problem += d2_high <= x2
problem += d3_high <= x3

# Demand constraints for medium demand
problem += d1_med <= 12
problem += d2_med <= 30
problem += d3_med <= 170
problem += d1_med <= x1
problem += d2_med <= x2
problem += d3_med <= x3

# Demand constraints for low demand
problem += d1_low <= 5
problem += d2_low <= 9
problem += d3_low <= 150
problem += d1_low <= x1
problem += d2_low <= x2
problem += d3_low <= x3

# Solve the problem
problem.solve()

# Output the results
print(f"First-Class Seats: {x1.varValue}")
print(f"Business-Class Seats: {x2.varValue}")
print(f"Economy-Class Seats: {x3.varValue}")
print(f"Expected Revenue: ${problem.objective.value():,.2f}")


First-Class Seats: 12.0
Business-Class Seats: 30.0
Economy-Class Seats: 121.0
Expected Revenue: $102,410.00


In [4]:
import random
from pulp import LpMaximize, LpProblem, LpVariable, value
import pandas as pd

# Function to simulate demand and solve the optimization problem
def simulate_demand():
    # Randomly select demand scenario
    scenario = random.choices(["High", "Medium", "Low"], weights=[40, 30, 30], k=1)[0]
    
    # Set demand limits based on scenario
    if scenario == "High":
        demand_limits = {"d1": 25, "d2": 60, "d3": 210}
    elif scenario == "Medium":
        demand_limits = {"d1": 12, "d2": 30, "d3": 170}
    else:  # Low demand
        demand_limits = {"d1": 5, "d2": 9, "d3": 150}
    
    # Define the optimization problem
    problem = LpProblem("Maximize_Revenue", LpMaximize)

    # Decision variables
    x1 = LpVariable("First_Class_Seats", lowBound=0, cat="Integer")
    x2 = LpVariable("Business_Class_Seats", lowBound=0, cat="Integer")
    x3 = LpVariable("Economy_Class_Seats", lowBound=0, cat="Integer")
    
    d1 = LpVariable("First_Class_Sold", lowBound=0, cat="Integer")
    d2 = LpVariable("Business_Class_Sold", lowBound=0, cat="Integer")
    d3 = LpVariable("Economy_Class_Sold", lowBound=0, cat="Integer")

    # Objective function: Revenue
    problem += 1600 * d1 + 1100 * d2 + 500 * d3

    # Capacity constraint
    problem += 2 * x1 + 1.5 * x2 + x3 <= 190

    # Demand constraints
    problem += d1 <= demand_limits["d1"]
    problem += d2 <= demand_limits["d2"]
    problem += d3 <= demand_limits["d3"]
    problem += d1 <= x1
    problem += d2 <= x2
    problem += d3 <= x3

    # Solve the problem
    problem.solve()

    # Return the scenario, seat allocations, and revenue
    return {
        "Scenario": scenario,
        "First_Class_Seats": x1.varValue,
        "Business_Class_Seats": x2.varValue,
        "Economy_Class_Seats": x3.varValue,
        "Revenue": value(problem.objective)
    }

# Monte Carlo simulation
num_simulations = 10
results = [simulate_demand() for _ in range(num_simulations)]

# Convert results to a DataFrame for analysis
results_df = pd.DataFrame(results)

# Analyze results
average_revenue = results_df["Revenue"].mean()
max_revenue = results_df["Revenue"].max()
min_revenue = results_df["Revenue"].min()

# Output results
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
simulation_summary = {
    "Average Revenue": f"${average_revenue:,.2f}",
    "Maximum Revenue": f"${max_revenue:,.2f}",
    "Minimum Revenue": f"${min_revenue:,.2f}"
}
results_df, simulation_summary


(     Scenario  First_Class_Seats  Business_Class_Seats  Economy_Class_Seats  \
 0         Low               13.0                   9.0                150.0   
 1      Medium               12.0                  30.0                121.0   
 2         Low               13.0                   9.0                150.0   
 3         Low               13.0                   9.0                150.0   
 4        High               25.0                  60.0                 50.0   
 5         Low               13.0                   9.0                150.0   
 6      Medium               12.0                  30.0                121.0   
 7        High               25.0                  60.0                 50.0   
 8      Medium               12.0                  30.0                121.0   
 9        High               25.0                  60.0                 50.0   
 10       High               25.0                  60.0                 50.0   
 11       High               25.0       